
Recopilamos datos históricos sobre caudales de ríos, precipitaciones, temperaturas, humedad y otros factores climáticos de diversas fuentes, incluyendo estaciones meteorológicas locales y bases de datos regionales. Se realizó un análisis exhaustivo de los datos para identificar patrones, tendencias estacionales y posibles problemas de calidad, como valores atípicos y datos faltantes. Además, se analizaron correlaciones entre las variables para comprender qué factores tenían mayor influencia en la disponibilidad de recursos hídricos

* CÓDIGO ESTACIÓN: Representa el Identificador único asignado a cada estación de monitoreo, que permite diferenciarlas dentro del sistema.

* NOMBRE ESTACIÓN: Muestra la denominación dada a la estación hidrológica, que facilita su identificación geográfica.

* FECHA: Muestra el día el mes y el año en el que se tomó la medición.

* NIVEL: Representa la altura del agua en el cauce o reservorio asociado a la estación, utilizada para evaluar la cantidad de agua disponible.

* PRECIPITACIÓN:Indica la cantidad de lluvia acumulada en un periodo de tiempo, medida en milímetros, crucial para entender los aportes hídricos.

* TEMPERATURA: Representa la medición de la temperatura ambiental en grados Celsius, que influye en procesos de evaporación y humedad del suelo.

* HUMEDAD: Indica el porcentaje de vapor de agua en el aire, clave para evaluar la capacidad de retención de agua en la atmósfera y su impacto en las precipitaciones.

* VELOCIDAD: Muestra la velocidad del viento en metros por segundo, que afecta la evaporación y dispersión de la humedad en el área de estudio.



# Autenticador Drive

In [1]:
# vincular colab con Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


#Librerias

In [11]:
# @title Instalar libreria
!pip install sweetviz
!pip install category_encoders

In [12]:
# importar librerias y modulos a utilizar
import os
import re
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import sweetviz as sw
import category_encoders as ce
from google.colab import files

#ML

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_auc_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.decomposition import PCA
import xgboost as xgb


#Balanceo
from imblearn.under_sampling import CondensedNearestNeighbour, EditedNearestNeighbours, InstanceHardnessThreshold, RepeatedEditedNearestNeighbours, OneSidedSelection
from imblearn.under_sampling import TomekLinks
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.cluster import MiniBatchKMeans
from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import NearMiss

#Algoritmos
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_text
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# Funciones

In [4]:
# @title Cargamos las funciones relacionadas
# Función para analizar la base de datos que devuelve cantidad de registros, valores únicos, total de vacíos, porcentaje de valores faltantes
def analizar_columnas(df):
  """
  Analiza las columnas de un DataFrame y devuelve información estadística.

  Parámetros:
  df (DataFrame): El DataFrame a analizar.

  Retorno:
  Un DataFrame con la información estadística de cada columna.

  Ejemplo:
  df_analizado = analizar_columnas(df)
  print(df_analizado)
  """
  info = []
  for columna in df.columns:
    unicos = df[columna].nunique()
    nulos = df[columna].isnull().sum()
    total = len(df)
    porcentaje_nulos = (nulos / total) * 100
    info.append({
      'Columna': columna,
      'Cantidad de registros': total,
      'Únicos': unicos,
      'Cantidad de nulos': nulos,
      'Porcentaje de nulos': porcentaje_nulos,
      'Registros relacionados con nulos': total - nulos
    })
  return pd.DataFrame(info)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Ruta de datos

In [13]:
/content/drive/MyDrive/INTELIGENCIA ARTIFICIAL/BASE DE DATOS Y PROYECTO FINAL/DATOS/BASE PACIENTES TRIAGE 2024.xlsx

SyntaxError: invalid decimal literal (<ipython-input-13-e10abb46308c>, line 1)

# Base de datos

In [10]:
# cargamos cada una de las bases de datos
precipitacion = pd.read_excel(path+"BASE PACIENTES TRIAGE 2024.xlsx")



NameError: name 'path' is not defined

# Limpieza de datos

In [ ]:
nivel.head(3)

In [ ]:
# limpiamos los datos, renombrando la columna Valor de cada base de datos por el valor correspondiente
nivel = nivel.rename(columns={'Valor': 'Nivel'})
precipitacion = precipitacion.rename(columns={'Valor': 'Precipitacion'})
temp = temp.rename(columns={'Valor': 'Temperatura'})
hum = hum.rename(columns={'Valor': 'Humedad'})
vel = vel.rename(columns={'Valor': 'Velocidad'})

In [ ]:
nivel.head(3)

In [ ]:
# muestra los datos únicos dentro de una variable
nivel.Latitud.unique()

In [ ]:
# después de analizar cuales columnas no nos interesan, las eliminamos
nivel.drop(columns=['Latitud', 'Longitud', 'Altitud','Categoria', 'Entidad', 'AreaOperativa', 'Departamento', 'Municipio',
                    'FechaInstalacion', 'FechaSuspension', 'IdParametro', 'Etiqueta','DescripcionSerie', 'Frecuencia',
                    'Grado','Calificador', 'NivelAprobacion'], inplace=True)

In [ ]:
nivel.head(3)

In [ ]:
# estandarizamos los formatos de fecha en todas las bases de datos
nivel['Fecha'] = pd.to_datetime(nivel['Fecha'], format='%Y-%m-%d %H:%M')
precipitacion['Fecha'] = pd.to_datetime(precipitacion['Fecha'], format='%m/%d/%Y')
temp['Fecha'] = pd.to_datetime(temp['Fecha'], format='%m/%d/%Y')
hum['Fecha'] = pd.to_datetime(hum['Fecha'], format='%m/%d/%Y')
vel['Fecha'] = pd.to_datetime(vel['Fecha'], format='%m/%d/%Y')

In [ ]:
nivel.info()

In [ ]:
# llevamos las variables que nos interesan a la base de datos principal usando como llave la variable Fecha
# copiamos la variable Precipitación de la base de datos precipitación a la base de datos nivel
df1 = pd.merge(nivel, precipitacion[['Fecha', 'Precipitacion']], how='left', on=['Fecha'])
# copiamos la variable Temperatura de la base de datos temp a la base de datos nivel
df2 = pd.merge(df1, temp[['Fecha', 'Temperatura']], how='left', on=['Fecha'])
# copiamos la variable Humedad de la base de datos hum a la base de datos nivel
df3 = pd.merge(df2, hum[['Fecha', 'Humedad']], how='left', on=['Fecha'])
# copiamos la variable Velocidad de la base de datos vel a la base de datos nivel
df4 = pd.merge(df3, vel[['Fecha', 'Velocidad']], how='left', on=['Fecha'])

In [ ]:
df4.head(3)

In [ ]:
df4.info()

In [ ]:
df4['day'] = pd.DatetimeIndex(df4['Fecha']).day
df4['month'] = pd.DatetimeIndex(df4['Fecha']).month
df4.head(32)

In [ ]:
# @title Verificar la forma de los datos
print(f"Forma de los datos (filas, columnas): {df4.shape}")

In [ ]:
# @title Verificar los tipos de datos de cada columna
print("\nTipos de datos por columna:")
print(df4.dtypes)

In [ ]:
#@title Contar los valores únicos por columna
print("\nValores únicos por columna:")
print(df4.nunique())

In [ ]:
#@title Verificar si hay datos faltantes
print("\nDatos faltantes por columna:")
print(df4.isnull().sum())

In [ ]:
# @title Verificación de los campos con relación a la composición de la base de dato almacenada en formato CSV
df_analizado = analizar_columnas(df4)
df_analizado

In [ ]:
# @title Realizamos la descriptiva estadística
df4.describe()

In [ ]:
# @title Análisis exploratorio de datos (EDA)
#Configurar las opciones de visualizacion Sweetviz
sw.config_parser.read_string("""
                              [Output_Defaults]
                              html_layout = widescreen
                              html_scale = 1.0
                              notebook_layout = widescreen
                              notebook_scale = 0.9
                              notebook_width = 100%
                              notebook_height = 700
                              [Layout]
                              show_logo = 0
                              """)
nombre = 'Cuencas'
advert_report = sw.analyze([df4, nombre]) # Realizamos el análisis de la base de datos inicial

#@markdown Guardar y mostrar reporte formato HTML
advert_report.show_html('EDA_df.html')

#@markdown Descarga y abre el reporte en una nueva pestaña del navegador
almacenar_archivo = input('Alamcenar archivo Si o No: ').title()
if almacenar_archivo == 'Si':
  files.download('/content/EDA_df.html')

In [ ]:
df4.info()

In [ ]:
# borramos los registros con datos faltantes de precipitación, temperatura y humedad
df4.dropna(subset=['Precipitacion', 'Temperatura', 'Humedad'], inplace=True) #, 'Velocidad'

In [ ]:
# codificamos las variables day y month

#Target encoding
#encoder = ce.TargetEncoder(cols=['day', 'month'])
#df4[['day', 'month']] = encoder.fit_transform(df4[['day', 'month']], df4['Nivel'])

#Leave one out encoding
encoder = ce.LeaveOneOutEncoder(cols=['day', 'month'])
df4[['day', 'month']] = encoder.fit_transform(df4[['day', 'month']], df4['Nivel'])

In [ ]:
df4.head(3)

In [ ]:
# separamos caracteristicas y etiquetas
X, y = df4[['Precipitacion', 'Temperatura', 'Humedad', 'day', 'month']].values, df4['Nivel'].values
print('Features:',X[:10], '\nLabels:', y[:10], sep='\n')

In [ ]:
#@title Seleccionar solo las columnas numéricas del DataFrame
numeric_cols = df4[['Precipitacion', 'Temperatura', 'Humedad', 'day', 'month']].select_dtypes(include=['number'])

# Calcular la matriz de correlación
conf_matrix = numeric_cols.corr()

# Mostrar la matriz de correlación
conf_matrix

In [ ]:
# @title Visualización de la correlación entre variables numéricas
plt.figure(figsize=(12, 10))
sns.heatmap(conf_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlación')
plt.show()

In [ ]:
#@title visualizar la varianza de las variables numericas con numeric_cols.var() en barras y en linea juntas

# Calcular la varianza de las columnas numéricas
variance = numeric_cols.var()

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
plt.bar(variance.index, variance.values)
plt.title('Varianza de las variables numéricas (Barras)')
plt.xlabel('Variables')
plt.ylabel('Varianza')
plt.xticks(rotation=90)
plt.show()

# Crear el gráfico de línea
plt.figure(figsize=(10, 6))
plt.plot(variance.index, variance.values)
plt.title('Varianza de las variables numéricas (Línea)')
plt.xlabel('Variables')
plt.ylabel('Varianza')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Dividimos los datos 70%-30% en datos de entrenamiento y datos de pruebas
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

print ('Datos de entrenamiento: %d rows\nDatos de prueba: %d rows' % (X_train.shape[0], X_test.shape[0]))

In [ ]:
# Entrenamiento del modelo

# Entrenamos el modelo de regresión lineal con los datos de entrenamiento
model = LinearRegression().fit(X_train, y_train)
print (model)

## Evaluación del modelo entrenado

In [ ]:
# Generamos las predicciones con los datos de prueba
predictions = model.predict(X_test)
np.set_printoptions(suppress=True)
print('Etiquetas predichas: ', np.round(predictions)[:10])
print('Etiquetas actuales : ',y_test[:10])

In [ ]:
# Muestra las gráficas generadas en la misma celda
%matplotlib inline

# Graficamos las predicciones vs los datos reales
plt.scatter(y_test, predictions)
plt.xlabel('Etiquetas actuales')
plt.ylabel('Etiquetas predichas')
plt.title('Predicción de nivel de cuencas')
# Sobreponemos la línea de regresión
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

In [ ]:
# calculamos las métricas para este modelo
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)

rmse = np.sqrt(mse)
print("RMSE:", rmse)

r2 = r2_score(y_test, predictions)
print("R2:", r2)

# Regresion polinomica

In [ ]:
# Entrenamiento del modelo de regresión polinómica

#Se define el grado del polinomio
poli_reg = PolynomialFeatures(degree = 2) #7

#Se transforman las características existentes en características de mayor grado
X_train_poli = poli_reg.fit_transform(X_train)
X_test_poli = poli_reg.fit_transform(X_test)

#Definimos el algoritmo a utilizar
pr = LinearRegression()

#Entrenamos el modelo
pr.fit(X_train_poli, y_train)

#Realizamos una predicción
Y_pred_pr = pr.predict(X_test_poli)

print('Valor de la pendiente o coeficiente "a":')
print(pr.coef_)

print('Valor de la intersección o coeficiente "b":')
print(pr.intercept_)

print('Precisión del modelo:')
print(pr.score(X_train_poli, y_train))

In [ ]:
# Calcular el R^2 en el conjunto de prueba
r2_test = r2_score(y_test, Y_pred_pr)
print('R^2 en el conjunto de prueba:', r2_test)

In [ ]:
# Entrenamiento del módelo de Random Forest Regressor usando los datos de entrenamiento
model = RandomForestRegressor().fit(X_train, y_train)
print (model, "\n")

# Evaluamos el modelo usando los datos de prueba
predictions = model.predict(X_test)

# Calculamos las métricas
# Error Cuadrático Medio (MSE):
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
# Raíz del Error Cuadrático Medio (RMSE):
rmse = np.sqrt(mse)
print("RMSE:", rmse)
# Coeficiente de Determinación (R²):
r2 = r2_score(y_test, predictions)
print("R2:", r2)

# Graficamos etiquetas predichas vs actuales
plt.scatter(y_test, predictions)
plt.xlabel('Etiquetas actuales')
plt.ylabel('Etiquetas predichas')
plt.title('Predicción de nivel de cuencas')

# sobreponemos la línea de regresión
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

In [ ]:
# Entrenamos el modelo Decision Tree Regressor con los datos de entrenamiento
model = DecisionTreeRegressor().fit(X_train, y_train)
print (model, "\n")

# Visualizamos el árbol del modelo
tree = export_text(model)
print(tree)

In [ ]:
# Evaluamos el modelo usando los datos de prueba
predictions = model.predict(X_test)

# Calculamos las métricas
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, predictions)
print("R2:", r2)

# Graficamos etiquetas predichas vs actuales
plt.scatter(y_test, predictions)
plt.xlabel('Etiquetas actuales')
plt.ylabel('Etiquetas predichas')
plt.title('Predicción de nivel de cuencas')

# sobreponemos la línea de regresión
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

In [ ]:
# Entrenamiento del modelo Gradient Boosting Regressor

# Entrenamos el modelo usando los datos de entrenamiento
model = GradientBoostingRegressor().fit(X_train, y_train)
print (model, "\n")

# Evaluamos el modelo usando los datos de prueba
predictions = model.predict(X_test)
# Calculamos las métricas
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, predictions)
print("R2:", r2)

# Graficamos etiquetas predichas vs actuales
plt.scatter(y_test, predictions)
plt.xlabel('Etiquetas actuales')
plt.ylabel('Etiquetas predichas')
plt.title('Predicción de nivel de cuencas')
# sobreponemos la línea de regresión
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()